In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Antibiotic_70K_patinets.csv")
df.head()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
0,1,2,1,2,1,50.00,1,1
1,2,25,9,16,16,36.00,1,5
2,3,6,0,6,6,0.00,1,4
3,4,7,6,6,1,85.71,1,4
4,5,4,3,2,1,75.00,1,3


## 문제 1

In [10]:
df.agg(["mean", "max", "min", "std", "skew", "kurt"])

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
mean,35000.500000,8.049771,3.774386,6.131871,4.275386,42.254031,0.021471,2.748700
max,70000.000000,526.000000,368.000000,397.000000,294.000000,100.000000,1.000000,7.000000
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
std,20207.403759,13.885638,8.795789,11.502711,7.950063,39.836986,0.144951,1.561020
skew,0.000000,7.884439,10.181053,8.288991,7.518696,0.274925,6.602827,0.311375
kurt,-1.200000,122.040109,215.833077,134.564408,112.589574,-1.503892,41.598519,-1.223271


## 문제 2

In [8]:
df_agg = df.groupby("age_5")["high_p"].mean().reset_index()
df_agg.loc[df_agg["high_p"].idxmax(), "age_5"]

7

## 문제 3

In [10]:
df.apply(lambda x: x.isna().sum())

ID             0
total_d        0
high_d         0
low_d          0
none_high_d    0
high_p         0
cdiff          0
age_5          0
dtype: int64

In [11]:
df.apply(lambda x: x.isna().sum()).sum()

0

In [12]:
d_mean = df["total_d"].mean()
d_sd = df["total_d"].std()

In [14]:
sum((df["total_d"] < (d_mean - 3 * d_sd)) | (df["total_d"] > (d_mean + 3 * d_sd)))

1258

## 문제 4

In [15]:
df[["high_p", "age_5"]].corr()

,high_p,age_5
high_p,1.000000,0.125046
age_5,0.125046,1.000000


## 문제 5

In [13]:
cdiff_0 = df.loc[df["cdiff"] == 0, "high_p"]
cdiff_1 = df.loc[df["cdiff"] == 1, "high_p"]

In [15]:
abs(cdiff_0.mean() - cdiff_1.mean())

4.791197866469155

In [16]:
from scipy.stats import ttest_ind
stat, p = ttest_ind(cdiff_0, cdiff_1)
print(stat)
print(p)

-4.613035361614422
3.975323449530875e-06


In [17]:
if p < 0.05:
    print("귀무가설 기각")
else:
    print("귀무가설 기각X")

귀무가설 기각


## 문제 6

In [40]:
from sklearn.linear_model import LogisticRegression

In [43]:
model = LogisticRegression(solver = "newton-cg").fit(df.drop(["cdiff", "none_high_d", "ID"], axis = 1), df["cdiff"])

In [55]:
df.drop(["cdiff", "none_high_d", "ID"], axis = 1).columns.values

array(['total_d', 'high_d', 'low_d', 'high_p', 'age_5'], dtype=object)

In [60]:
df_model = pd.DataFrame({"var": df.drop(["cdiff", "none_high_d", "ID"], axis = 1).columns.values,
                         "coef": model.coef_[0]})
df_model

,var,coef
0,total_d,0.011335
1,high_d,-0.003268
2,low_d,-0.010503
3,high_p,0.000679
4,age_5,0.531175


In [61]:
df_model.loc[df_model["coef"] < 0, "var"]

1    high_d
2     low_d
Name: var, dtype: object